In [2]:
%matplotlib inline
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

# torch.cuda.is_available()

In [3]:
# Something to do with diplsays??
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython: from IPython import display

In [4]:
# Example visualisation of environment
env = gym.make('CartPole-v0')
env.reset()
done = False
for _ in range(500):
    env.render()
    env.step(env.action_space.sample())
done = True
env.close()


C:\Users\neale\anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [5]:
#Classes

class DQN(nn.Module):

    def __init__(self, h, w):
        super(DQN, self).__init__()
        
        self.fc1 = nn.Linear(h*w*3, 24)
        self.fc2 = nn.Linear(24,32)
        self.out = nn.Linear(32,2)
        
    def forward(self, t):
        t = t.flatten(start_dim=1)
        t = F.relu(self.fc1(t))
        t = F.relu(self.fc2(t))
        t = self.out(t)
        return t
        
        
class ReplayMemory():
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.push_count = 0
        
    def push(self, experience):
        if len(self.memory) < self.capacity: # adds new experience to replay memory
            self.memory.append(experience) 
        else:
            self.memory[self.push_count % self.capacity] = experience #if replay memory is over capacity 
        self.push_count +=1                                           #it will start re-writing over memory
        
    def sample(self,batch_size): #returns random batch
        return random.sample(self.memory, batch_size)
    
    def can_provide_sample(self,batch_size): #returns boolean if able to start sampling from memory
        return len(self.memory) >= batch_size
    
class EpsilonGreedyStrategy():
    def __init__(self,start,end,decay):
        self.start = start
        self.end = end
        self.decay = decay
    
    def get_exploration_rate(self,current_step): # N(t) = Ne^-ht
        return self.end + (self.start - self.end) *  \
            math.exp(-1. *current_step * self.decay)
    
class Agent():
    def __init__(self, strategy, num_actions, device):
        self.current_step = 0
        self.strategy = strategy
        self.num_actions = num_actions
        self.device = device
        
    def select_action(self,state,policy_net):
        rate = strategy.get_exploration_rate(self.current_step)
        self.current_step += 1
        
        if rate > random.random():
            action = random.randrange(self.num_actions)
            return torch.tensor([action]).to(self.device) # explore      
        else:
            with torch.no_grad():
                return policy_net(state).argmax(dim=1).to(self.device) # exploit #exploit
            
class CartPoleEnvManager():
    def __init__(self,device):
        self.device = device
        self.env = gym.make('CartPole-v0').unwrapped
        self.env.reset()
        self.current_screen = None
        self.done = False
        
    def reset(self):
        self.env.reset()
        self.current_screen = None
        
    def close(self):
        self.env.close()
        
    def render(self, mode='human'):
        return self.env.render(mode)
    
    def num_actions_available(self):
        return self.env.action_space.n
    
    def take_action(self, action):        
        _, reward, self.done, _ = self.env.step(action.item())
        return torch.tensor([reward], device=self.device)
    
    def just_starting(self):
        return self.current_screen is None
    
    def get_state(self): # We are comparing 2 screen
        if self.just_starting() or self.done: # if start of episode or end of episode:
            self.current_screen = self.get_processed_screen()
            black_screen = torch.zeros_like(self.current_screen)
            return black_screen
        else:
            s1 = self.current_screen
            s2 = self.get_processed_screen()
            self.current_screen = s2
            return s2-s1
        
    def get_screen_height(self):
        screen = self.get_processed_screen()
        return screen.shape[2]
          
    def get_screen_width(self):
        screen = self.get_processed_screen()
        return screen.shape[3]
    
    def get_processed_screen(self):
        screen = self.render('rgb_array').transpose((2,0,1))
        screen = self.crop_screen(screen)
        return self.transform_screen_data(screen)
        # be careful, this function doens't have call a close() after render()
    
    def crop_screen(self, screen):
        # strip top and bottom of screen
        screen_height = screen.shape[1]
        top = int(screen_height * 0.4)
        bottom = int(screen_height * 0.8)
        screen = screen[:, top:bottom, :]
        return screen
    
    def transform_screen_data(self,screen):
        # Use torchvision package to compose image transforms
        screen = np.ascontiguousarray(screen, dtype =np.float32) / 255
        screen = torch.from_numpy(screen)
        
        resize = T.Compose([
            T.ToPILImage(),
            T.Resize((40,90)),
            T.ToTensor()
        ])
        
        return resize(screen).unsqueeze(0).to(self.device) #unsqueeze adds a batch dim

class QValues():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    @staticmethod
    def get_current(policy_net, states, actions):
        return policy_net(states).gather(dim=1, index=actions.unsqueeze(-1))
    
    @staticmethod        
    def get_next(target_net, next_states):                
        final_state_locations = next_states.flatten(start_dim=1) \
            .max(dim=1)[0].eq(0).type(torch.bool)
        non_final_state_locations = (final_state_locations == False)
        non_final_states = next_states[non_final_state_locations]
        batch_size = next_states.shape[0]
        values = torch.zeros(batch_size).to(QValues.device)
        values[non_final_state_locations] = target_net(non_final_states).max(dim=1)[0].detach()
        return values
    
print('done')

done


In [6]:
# functions
def plot(values, moving_avg_period):
    plt.figure(2)
    plt.clf()
    plt.title('Training..')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(values)
    moving_avg = get_moving_average(moving_avg_period,values)
    plt.plot(moving_avg)
    
    plt.pause(0.001)
    print("Episode", len(values), "\n", moving_avg_period, "episode moving avg:", moving_avg[-1])
    if is_ipython: display.clear_output(wait=True)
    
        
def get_moving_average(period, values):
    values = torch.tensor(values, dtype=torch.float)
    if len(values) >= period:
        moving_avg = values.unfold(dimension=0, size=period, step=1) \
            .mean(dim=1).flatten(start_dim=0)
        moving_avg = torch.cat((torch.zeros(period-1), moving_avg))
        return moving_avg.numpy()
    else:
        moving_avg = torch.zeros(len(values))
        return moving_avg.numpy()
        
Experience = namedtuple( #Experience item
    'Experience',
    ('state','action','next_state','reward')
)    

def extract_tensors(experiences):
    # Convert batch of Experiences to Experience of batches
    batch = Experience(*zip(*experiences))

    t1 = torch.cat(batch.state)
    t2 = torch.cat(batch.action)
    t3 = torch.cat(batch.reward)
    t4 = torch.cat(batch.next_state)

    return (t1,t2,t3,t4)

In [7]:

# Main program

# Hyperparams
batch_size = 5
gamma = 0.999
eps_start = 0.9
eps_end = 0.05
eps_decay = 0.005
target_update = 10
memory_size = 10000
lr = 0.001
num_episodes = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

em = CartPoleEnvManager(device)
strategy = EpsilonGreedyStrategy(eps_start, eps_end, eps_decay)
agent = Agent(strategy, em.num_actions_available(), device)
memory = ReplayMemory(memory_size)

policy_net = DQN(em.get_screen_height(), em.get_screen_width()).to(device)
target_net = DQN(em.get_screen_height(), em.get_screen_width()).to(device)

target_net.load_state_dict(policy_net.state_dict()) # Copies Weights and biases to target_network
target_net.eval()

optimizer = optim.Adam(params=policy_net.parameters(), lr=lr)


# Training loop
episode_durations = []

for episode in range(num_episodes):
    em.reset()
    state = em.get_state()
    
    for timestep in count():
        action = agent.select_action(state, policy_net)
        reward = em.take_action(action)
        next_state = em.get_state()
        memory.push(Experience(state, action, next_state, reward))
        state = next_state # Agent has taken an action, got new_state and reward and stores in memory
        
        if memory.can_provide_sample(batch_size):
            experiences = memory.sample(batch_size)
            states, actions, rewards, next_states = extract_tensors(experiences) #groups and extracts data by category e.g creates a states tensor, rewards tensor ...
            break
            
"""
            current_q_values = QValues.get_current(policy_net,states,actions)
            next_q_values = QValues.get_next(target_net, next_states)
            target_q_values = (next_q_values * gamma) + rewards
            
            loss = F.mse_loss(current_q_values, target_q_values.unsqueeze(1))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        if em.done:
            episode_durations.append(timestep)
            plot(episode_durations, 100)
            break
            
    if episode % target_update == 0:
        target_net.load_state_dict(policy_net.state_dict())

"""
em.close()
            

In [23]:
rewards.shape

torch.Size([5])

In [9]:
em.reset()
env = em.get_state()

em.close()

env.shape


torch.Size([1, 3, 40, 90])

In [17]:
policy_net(env)

tensor([[-0.1970,  0.1072]], device='cuda:0', grad_fn=<AddmmBackward>)

In [9]:
# Visualise model

# params
episodes = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
episode_duration = []

# main loop
em = CartPoleEnvManager(device)

for i in range(episodes):
    em.reset()
    state = em.get_state()
    
    for steps in count(): # eventually replace with count() ? or i suppose just use a while loop?
        em.render()
        action = model(state).argmax(dim=1).to(device)
        em.take_action(action)
        next_state = em.get_state()
        
        if em.done:
            episode_duration.append(steps)
            break
            
        state = next_state

print(" Mean episode length", np.mean(episode_duration))
em.close()
           

AttributeError: 'DQN' object has no attribute 'bn1'

In [3]:
FILE = 'DQN_v1.pth'

In [33]:
# export model
torch.save(policy_net, FILE)

In [4]:
# import model
model = torch.load(FILE)
model.eval()



AttributeError: Can't get attribute 'DQN' on <module '__main__'>

In [28]:
1+2

3

In [ ]:
em.close()